## Multitask-learning

[ref](https://towardsdatascience.com/multi-task-learning-for-computer-vision-classification-with-keras-36c52e6243d2)
- 讀取資料
- 創建模型(output修改)
- 訓練loss、metrics增加
- 訓練

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

In [3]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [4]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [5]:
y_train = keras.utils.to_categorical(y_train, num_classes=10, dtype="float32")
y_test = keras.utils.to_categorical(y_test, num_classes=10, dtype="float32")

In [6]:
# modeling

def get_multitask_model():
    inputs = keras.Input(shape=x_train.shape[1:])
    x = keras.layers.Conv2D(filters=32, kernel_size=32, padding='same')(inputs)
    x = keras.layers.MaxPooling2D(padding='same')(x)
    x = keras.layers.Conv2D(filters=32, kernel_size=32, padding='same')(x)
    x = keras.layers.MaxPooling2D(padding='same')(x)
    x = keras.layers.Flatten()(x)
    
    output1 = keras.layers.Dense(units=10, activation='softmax', name='task1')(x)
    output2 = keras.layers.Dense(units=10, activation='softmax', name='task2')(x)
    
    return keras.Model(inputs, [output1, output2])

In [7]:
model = get_multitask_model()

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 28, 28, 32)   32800       ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 14, 14, 32)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 14, 14, 32)   1048608     ['max_pooling2d[0][0]']          
                                                                                              

In [9]:
gamma = 0.3
model.compile(optimizer='adam',
                  loss={'task1': 'categorical_crossentropy', 
                        'task2': 'categorical_crossentropy'},
                  loss_weights={'task1': gamma, 
                                'task2': 1 - gamma}, 
                  metrics=['accuracy'])

In [10]:
model.fit(x_train, y=[y_train, y_train], batch_size=32, epochs=2, validation_data=(x_test, [y_test, y_test]))

Epoch 1/2
1875/1875 [==============================] - 735s 392ms/step - loss: 0.2795 - task1_loss: 0.2768 - task2_loss: 0.2806 - task1_accuracy: 0.9269 - task2_accuracy: 0.9286 - val_loss: 0.2001 - val_task1_loss: 0.1881 - val_task2_loss: 0.2053 - val_task1_accuracy: 0.9588 - val_task2_accuracy: 0.9555
Epoch 2/2
1875/1875 [==============================] - 714s 381ms/step - loss: 0.2515 - task1_loss: 0.2497 - task2_loss: 0.2523 - task1_accuracy: 0.9555 - task2_accuracy: 0.9541 - val_loss: 0.3249 - val_task1_loss: 0.3378 - val_task2_loss: 0.3194 - val_task1_accuracy: 0.9481 - val_task2_accuracy: 0.9475
